In [47]:
from datetime import *
from dateutil import *
try:
    from control import *
    from control import matlab
    from matplotlib.pyplot import *
    from pandas import *
    from numpy import *
    from numpy.random import *
    from scipy.optimize import *
except:
    import pip
    pip.main("install control matplotlib numpy pandas scipy slycot".split())
    raise Exception("try again")

In [2]:
UA = 500.0 # BTU/degF.h
CA = 5000.0 # BTU/degF
UM = 1000.0 # BTU/degF.h
CM = 10000.0 # BTU/degF
UO = 500.0 # BTU/degF.h
m = 10000.0 # BTU/h
QI = +1*3412.0 # BTU/h
QS = +1*3412.0 # BTU/h

In [3]:
TS = 72.0 # degF
TO = 95.0 # degF
DB = 1.0 # degF

In [4]:
TE = QI/UA + QS/UO + TO
if TO < 20:
    mode = "AUX"
    eta = +1.0 * 3412
elif abs(TE-TS) < DB:
    mode = "VENT"
    eta = +0.1 * 3412
elif TE > TS:
    mode = "COOL"
    eta = -3.0 * 3412
else:
    mode = "HEAT"
    eta = +4.0 * 3412
TE,mode

(108.648, 'COOL')

In [5]:
A = [[-(UM+UA)/CA, UM/CA, 1/CA],
     [UM/CM, -(UM+UO)/CM, 0],
     [-m*(UM+UA)/CA, m*UM/CA, 0]
    ]
B = [[UA/CA, 1, 1/CA, 0],
     [UO/CM, 1, 0, 1/CM],
     [m*UA/CA, 0, m/CA, 0]
    ]
C = [[0, 0, -1/eta]]
D = [[0, 0, 1/3412.0, 0]]

In [59]:
H = c2d(ss(A,B,C,D,timeunit='h'),1);H

StateSpace(array([[ 5.19114296e-01,  3.08907577e-01,  1.56842468e-04],
       [ 7.22564325e-02,  8.74091012e-01,  8.21973558e-06],
       [-2.27043967e+03,  1.44512865e+03,  7.46158263e-01]]), array([[ 1.71978127e-01,  9.24407054e-01,  3.29936783e-04,
         1.40194712e-05],
       [ 5.36525553e-02,  9.73955071e-01,  1.40194712e-05,
         9.32856393e-05],
       [ 8.25311020e+02, -4.47235127e+02,  1.56842468e+00,
         8.21973558e-02]]), array([[0.00000000e+00, 0.00000000e+00, 9.76944119e-05]]), array([[0.        , 0.        , 0.00029308, 0.        ]]), 1)

In [160]:
X0 = [randint(0,UA)*2,1/randint(1,CA*2),randint(0,UM*2),1/randint(1,CM*2),randint(1,UO*2),randint(1,m*2),-1/randint(1,-eta*2)]
list(map(lambda x: round(x),X0))

[126, 0, 1572, 0, 846, 4141, 0]

In [155]:
w = [2,1,2,0]
def error(X):
    UA = X[0] # BTU/degF.h
    CA = X[1] # BTU/degF
    UM = X[2] # BTU/degF.h
    CM = X[3] # BTU/degF
    UO = X[4] # BTU/degF.h
    m = X[5] # BTU/h
    eta = X[6] # pu
    A = [[-(UM+UA)/CA, UM/CA, 1/CA],
         [UM/CM, -(UM+UO)/CM, 0],
         [-m*(UM+UA)/CA, m*UM/CA, 0]
        ]
    B = [[UA/CA, 1, 1/CA, 0],
         [UO/CM, 1, 0, 1/CM],
         [m*UA/CA, 0, m/CA, 0]
        ]
    C = [[0, 0, -1/eta]]
    D = [[0, 0, 1/3412.0, 0]]
    G = c2d(ss(A,B,C,D,timeunit='h'),1)
    return  w[0]*linalg.norm(H.A-G.A,2) + \
            w[1]*linalg.norm(H.B-G.B,2) + \
            w[2]*linalg.norm(H.C-G.C,2) + \
            w[3]*linalg.norm(H.D-G.D,2)
res = minimize(error,X0,method="CG",options={"disp":False})
X = [res.x[0].round(1),res.x[1].round(1),res.x[2].round(1),res.x[3].round(1),res.x[4].round(1),res.x[5].round(1)]
DataFrame([X,[UA,CA,UM,CM,UO,m,eta]],columns=["UA","CA","UM","CM","UO","m","eta"])

,UA,CA,UM,CM,UO,m,eta
0,177.2,1425.2,240.0,-435.2,-31.7,11796.8,NaN
1,500.0,5000.0,1000.0,10000.0,500.0,10000.0,-10236.0


In [156]:
t,y = step_response(H)
def error(x):
    x0 = UA; x1=1/CA; x2=UM; x3=1/CM; x4=UO; x5=m; x6=1/eta
    A = [[-x0*x1-x2*x1, x2*x1, x1],
         [x2*x3, -x2*x3-x4*x3, 0],
         [-x5*x0*x1-x5*x2*x1, x5*x2*x1, 0]
        ]
    B = [[x0*x1, 1, x1, 0],
         [x4*x3, 1, 0, x3],
         [x5*x0*x1, 0, x5*x1, 0]
        ]
    C = [[0, 0, -x6]]
    D = [[0, 0, 1/3412.0, 0]]
    G = c2d(ss(A,B,C,D,timeunit='h'),1)
    _,yy = step_response(G,t)
    return linalg.norm(yy[0,0]-y[0,0],2)
res = minimize(error,X0,method="CG",options={"disp":False})
X = [res.x[0],1/res.x[1],res.x[2],1/res.x[3],res.x[4],res.x[5],1/res.x[6]]
DataFrame([X,[UA,CA,UM,CM,UO,m,eta]],columns=["UA","CA","UM","CM","UO","m","eta"])

,UA,CA,UM,CM,UO,m,eta
0,76.0,1512.0,791.0,6843.0,243.0,11797.0,-12757.0
1,500.0,5000.0,1000.0,10000.0,500.0,10000.0,-10236.0


In [157]:
w = [10,1,1,1]
def error(x):
    x0 = UA; x1=1/CA; x2=UM; x3=1/CM; x4=UO; x5=m; x6=1/eta
    A = [[-x0*x1-x2*x1, x2*x1, x1],
         [x2*x3, -x2*x3-x4*x3, 0],
         [-x5*x0*x1-x5*x2*x1, x5*x2*x1, 0]
        ]
    B = [[x0*x1, 1, x1, 0],
         [x4*x3, 1, 0, x3],
         [x5*x0*x1, 0, x5*x1, 0]
        ]
    C = [[0, 0, -x6]]
    D = [[0, 0, 1/3412.0, 0]]
    G = c2d(ss(A,B,C,D,timeunit='h'),1)
    return  w[0]*linalg.norm(H.A-G.A,2) + \
            w[1]*linalg.norm(H.B-G.B,2) + \
            w[2]*linalg.norm(H.C-G.C,2) + \
            w[3]*linalg.norm(H.D-G.D,2)
res = minimize(error,X0,method="CG",options={"disp":False})
X = [res.x[0],1/res.x[1],res.x[2],1/res.x[3],res.x[4],res.x[5],1/res.x[6]]
DataFrame([X,[UA,CA,UM,CM,UO,m,eta]],columns=["UA","CA","UM","CM","UO","m","eta"])

,UA,CA,UM,CM,UO,m,eta
0,76.0,1512.0,791.0,6843.0,243.0,11797.0,-12757.0
1,500.0,5000.0,1000.0,10000.0,500.0,10000.0,-10236.0


In [158]:
t,y = step_response(H)
w = [10,1,1,1]
def error(x):
    x0 = UA; x1=1/CA; x2=UM; x3=1/CM; x4=UO; x5=m; x6=1/eta
    A = [[-x0*x1-x2*x1, x2*x1, x1],
         [x2*x3, -x2*x3-x4*x3, 0],
         [-x5*x0*x1-x5*x2*x1, x5*x2*x1, 0]
        ]
    B = [[x0*x1, 1, x1, 0],
         [x4*x3, 1, 0, x3],
         [x5*x0*x1, 0, x5*x1, 0]
        ]
    C = [[0, 0, -x6]]
    D = [[0, 0, 1/3412.0, 0]]
    G = c2d(ss(A,B,C,D,timeunit='h'),1)
    _,yy = step_response(G,t)
    return  w[0]*linalg.norm(H.A-G.A,2) + \
            w[1]*linalg.norm(H.B-G.B,2) + \
            w[2]*linalg.norm(H.C-G.C,2) + \
            w[3]*linalg.norm(H.D-G.D,2) + \
            linalg.norm(yy[0,0]-y[0,0],2)
res = minimize(error,X0,method="CG",options={"disp":False})
X = [res.x[0],1/res.x[1],res.x[2],1/res.x[3],res.x[4],res.x[5],1/res.x[6]]
DataFrame([X,[UA,CA,UM,CM,UO,m,eta]],columns=["UA","CA","UM","CM","UO","m","eta"])

,UA,CA,UM,CM,UO,m,eta
0,76.0,1512.0,791.0,6843.0,243.0,11797.0,-12757.0
1,500.0,5000.0,1000.0,10000.0,500.0,10000.0,-10236.0
